<a href="https://colab.research.google.com/github/sahankrt20/AI-BOOTCAMP-PROJECT/blob/main/stock_market_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf

In [15]:

tsla_data = yf.download(tickers='TSLA',
                   start='2020-01-01',
                   end='2024-12-31')


[*********************100%***********************]  1 of 1 completed


In [16]:
print(tsla_data.head)

<bound method NDFrame.head of Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2020-01-02   28.684000   28.713333   28.114000   28.299999  142981500
2020-01-03   29.534000   30.266666   29.128000   29.366667  266677500
2020-01-06   30.102667   30.104000   29.333332   29.364668  151995000
2020-01-07   31.270666   31.441999   30.224001   30.760000  268231500
2020-01-08   32.809334   33.232666   31.215334   31.580000  467164500
...                ...         ...         ...         ...        ...
2024-12-23  430.600006  434.510010  415.410004  431.000000   72698100
2024-12-24  462.279999  462.779999  435.140015  435.899994   59551800
2024-12-26  454.130005  465.329987  451.019989  465.160004   76366400
2024-12-27  431.660004  450.000000  426.500000  449.519989   82666800
2024-12-30  417.410004  427.000000  415.750000  419.399994  

In [17]:
tsla_data = tsla_data.reset_index()
tsla_data

Price,Date,Close,High,Low,Open,Volume
Ticker,,TSLA,TSLA,TSLA,TSLA,TSLA
0,2020-01-02,28.684000,28.713333,28.114000,28.299999,142981500
1,2020-01-03,29.534000,30.266666,29.128000,29.366667,266677500
2,2020-01-06,30.102667,30.104000,29.333332,29.364668,151995000
3,2020-01-07,31.270666,31.441999,30.224001,30.760000,268231500
4,2020-01-08,32.809334,33.232666,31.215334,31.580000,467164500
...,...,...,...,...,...,...
1252,2024-12-23,430.600006,434.510010,415.410004,431.000000,72698100
1253,2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800
1254,2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400


In [14]:
tsla_data.columns

MultiIndex([( 'Close', 'TSLA'),
            (  'High', 'TSLA'),
            (   'Low', 'TSLA'),
            (  'Open', 'TSLA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])

In [13]:

AMZN_data = yf.download(tickers="AMZN",
                   start='2020-01-01',
                   end='2024-12-31')
print(AMZN_data.head)

[*********************100%***********************]  1 of 1 completed

<bound method NDFrame.head of Price            Close        High         Low        Open    Volume
Ticker            AMZN        AMZN        AMZN        AMZN      AMZN
Date                                                                
2020-01-02   94.900497   94.900497   93.207497   93.750000  80580000
2020-01-03   93.748497   94.309998   93.224998   93.224998  75288000
2020-01-06   95.143997   95.184502   93.000000   93.000000  81236000
2020-01-07   95.343002   95.694504   94.601997   95.224998  80898000
2020-01-08   94.598503   95.550003   94.321999   94.902000  70160000
...                ...         ...         ...         ...       ...
2024-12-23  225.059998  226.880005  223.899994  225.009995  28070000
2024-12-24  229.050003  229.139999  226.130005  226.940002  15007500
2024-12-26  227.050003  228.500000  226.669998  228.500000  16146700
2024-12-27  223.750000  226.029999  220.899994  225.600006  27367100
2024-12-30  221.300003  223.000000  218.429993  220.059998  28321200

[12

In [18]:
AMZN_data=AMZN_data.reset_index()

In [19]:
AMZN_data

Price,Date,Close,High,Low,Open,Volume
Ticker,,AMZN,AMZN,AMZN,AMZN,AMZN
0,2020-01-02,94.900497,94.900497,93.207497,93.750000,80580000
1,2020-01-03,93.748497,94.309998,93.224998,93.224998,75288000
2,2020-01-06,95.143997,95.184502,93.000000,93.000000,81236000
3,2020-01-07,95.343002,95.694504,94.601997,95.224998,80898000
4,2020-01-08,94.598503,95.550003,94.321999,94.902000,70160000
...,...,...,...,...,...,...
1252,2024-12-23,225.059998,226.880005,223.899994,225.009995,28070000
1253,2024-12-24,229.050003,229.139999,226.130005,226.940002,15007500
1254,2024-12-26,227.050003,228.500000,226.669998,228.500000,16146700
